### Create API gateway for get image function

In [1]:
import nuclio

In [2]:
%run set_env.ipynb

In [3]:
# nuclio: start-code

In [4]:
import urllib3
import requests
import json
import time
import os

# HACK for systems without production cert - shut up the no cert verification WARN
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [5]:
API_GATEWAY_NAME = f"{os.getenv('API_GATEWAY')}-{os.getenv('PROJECT')}"
API_GATEWAY_HOST = f"{API_GATEWAY_NAME}.{os.getenv('IGZ_NAMESPACE_DOMAIN')}"
API_FUNCTION = f"{os.getenv('PROJECT')}-{os.getenv('GET_IMAGE_FUNCTION')}"
DASHBOARD_URL = f"https://dashboard.{os.getenv('IGZ_NAMESPACE_DOMAIN')}"

In [6]:
session = requests.Session()
session.auth = (os.getenv("V3IO_USERNAME"), os.getenv("IGZ_AUTH"))
auth = session.post(f'{DASHBOARD_URL}/api/sessions', verify=False)

session_id = auth.json()['data']['id']
print(f'Session opened, your session ID is: {session_id}')

Session opened, your session ID is: b0ba8e2a-5e49-468c-b07e-215458233743


In [7]:
request = {
   'data':{
      'attributes':{
         'meta':{
            'name': os.getenv("API_GATEWAY"),
             "labels": {
                "entries": [
                  {
                    "value": os.getenv("V3IO_USERNAME"),
                    "key": "iguazio.com/username"
                  },
                  {
                    "value": os.getenv("PROJECT"),
                    "key": "project-name"
                  }
                ]
            },
         },
         'spec':{
            'authentication_mode':'none',
            'host': API_GATEWAY_HOST,
            'upstreams':[
               {
                  'kind':'nucliofunction',
                  'nucliofunction':{
                     'name': API_FUNCTION,
                  }
               },
            ],
            'description':f'Get image from "{os.getenv("TAGGED_VIDEO_STREAM")}"',
            'name': os.getenv("API_GATEWAY"),
         },
      },
      'type':'api_gateway'
   },
}

In [8]:
def handler(context, event):
    response = session.post(f'{DASHBOARD_URL}/api/api_gateways', json=request, headers={'x-igz-project-name': os.getenv("PROJECT")})
    if response.status_code == 409:
        print(f'API Gateway with name {API_GATEWAY_NAME} already exists (That\'s OK!)')
    else:
        print(json.dumps(response.json(), indent=2))

    print('Done creating API gateway!')

In [9]:
# nuclio: end-code

In [10]:
import os

In [11]:
# nuclio: ignore
# converts the notebook code to deployable function with configurations
from mlrun import code_to_function, mount_v3io, mlconf
import os

fn = code_to_function('create-api-gateway', kind='job', handler="handler", project="objectrecognition")

In [12]:
# set the API/trigger, attach the home dir to the function
# fn.with_http(workers=1).apply(mount_v3io())
fn.spec.build.base_image = 'mlrun/mlrun'
# fn.spec.base_spec['spec']['build']['baseImage'] = 'python:3.6-jessie'
# fn.spec.build.commands = ['pip install requests']
fn.spec.min_replicas = 1
fn.spec.max_replicas = 1
fn.set_env('API_GATEWAY', API_GATEWAY)
fn.set_env('PROJECT', PROJECT)
fn.set_env('GET_IMAGE_FUNCTION', GET_IMAGE_FUNCTION)
fn.set_env('V3IO_USERNAME', os.getenv('V3IO_USERNAME'))
fn.set_env('IGZ_AUTH', IGZ_AUTH)
fn.set_env('IGZ_NAMESPACE_DOMAIN', os.getenv('IGZ_NAMESPACE_DOMAIN'))
fn.set_env('TAGGED_VIDEO_STREAM', TAGGED_VIDEO_STREAM)
fn.apply(mount_v3io())

In [13]:
# print(fn.to_yaml())

In [15]:
# nuclio: ignore
# addr = fn.deploy(project='objectrecognition')
# addr = fn.deploy(with_mlrun=False)

In [17]:
fn.run(artifact_path=mlconf.dbpath)

> 2020-10-26 21:21:38,915 [info] starting run create-api-gateway-handler uid=50982609355649cb9470a396ca8d2aa9  -> http://mlrun-api:8080
> 2020-10-26 21:21:39,077 [info] Job is running in the background, pod: create-api-gateway-handler-sq68s
Session opened, your session ID is: b7ca8d79-ab76-4859-b981-f65b71d34601
{
  "included": [],
  "meta": {
    "ctx": "12859867783551689999"
  },
  "data": {
    "type": "job",
    "id": "2812463b-5b5e-4085-bf25-2ad3c35efc83"
  }
}
Done creating API gateway!
> 2020-10-26 21:21:43,619 [info] run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
objectrecognition,...ca8d2aa9,0,Oct 26 21:21:43,completed,create-api-gateway-handler,v3io_user=nickskind=jobowner=nickshost=create-api-gateway-handler-sq68s,,,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 50982609355649cb9470a396ca8d2aa9 --project objectrecognition , !mlrun logs 50982609355649cb9470a396ca8d2aa9 --project objectrecognition
> 2020-10-26 21:21:45,194 [info] run executed, status=completed


In [15]:
!curl $addr